In [ ]:
import torch
torch.__version__

torch.mps.is_available() # For Mac OS
print(torch.backends.mps.is_available())
# Mac has an Apple Silicon chip, to accelerate PyTorch code.

True


In [15]:
# 1. Tensors are data-containers for array-like
tensor0d = torch.tensor(1) # 0d Tensor
tensor1d = torch.tensor([1, 2, 3]) # 1D Tensor
tensor2d = torch.tensor([[1, 2, 3], [3, 4, 4]]) # 2D Tensor, From Nested Python List
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[1, 6], [2, 9]]])

# 2. Tensor DataTypes
print(tensor1d.dtype)

# Float Data Type
floatvec = torch.tensor([1.0, 3.0, 5.0, 6.9])
print(floatvec.dtype) # 32-bit
# A 32 bit offers sufficient precision, consume less memory & resources
# Most GPU Arch are optimized for 32-Bits computations.
# Hence Speed Up Model Trainig & inference

# Possible to change the precision using `.to` method
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

# 3 Tensor Operations
print(tensor2d)
print(tensor2d.shape) # Tensor has 2 rows and 3 columns
tensor2d_reshape = tensor2d.reshape(3, 2)
print(tensor2d_reshape)
print(tensor2d.shape)

tensor2d.view(3, 2) # Most common way to reshape

tensor2d.T # Transpose the tensor, flipping it across its diagonal

tensor2d.matmul(tensor2d.T) # matmul or @
tensor2d @ tensor2d.T

torch.int64
torch.float32
torch.float32
tensor([[1, 2, 3],
        [3, 4, 4]])
torch.Size([2, 3])
tensor([[1, 2],
        [3, 3],
        [4, 4]])
torch.Size([2, 3])


tensor([[14, 23],
        [23, 41]])